In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Download the data

In [18]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [19]:
concrete_data.shape

(1030, 9)

In [20]:
#Let's check the dataset for any missing values.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [21]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Split data into predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [22]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [23]:
#Let's do a quick sanity check of the predictors and the target dataframes.
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [24]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [25]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [26]:
#Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

## Build a Neural Network

In [27]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))

    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)

In [29]:
# build the model
model = regression_model()

## Train and Test the Network

In [30]:
# build the model
model.fit( X_test, y_test,epochs=50, verbose=2)

Epoch 1/50
10/10 - 0s - loss: 5151.7354
Epoch 2/50
10/10 - 0s - loss: 1570.0886
Epoch 3/50
10/10 - 0s - loss: 828.0478
Epoch 4/50
10/10 - 0s - loss: 844.9077
Epoch 5/50
10/10 - 0s - loss: 743.4565
Epoch 6/50
10/10 - 0s - loss: 661.8229
Epoch 7/50
10/10 - 0s - loss: 629.7221
Epoch 8/50
10/10 - 0s - loss: 591.5318
Epoch 9/50
10/10 - 0s - loss: 558.5214
Epoch 10/50
10/10 - 0s - loss: 528.9919
Epoch 11/50
10/10 - 0s - loss: 502.2093
Epoch 12/50
10/10 - 0s - loss: 477.5219
Epoch 13/50
10/10 - 0s - loss: 456.1164
Epoch 14/50
10/10 - 0s - loss: 435.7915
Epoch 15/50
10/10 - 0s - loss: 417.8751
Epoch 16/50
10/10 - 0s - loss: 400.9782
Epoch 17/50
10/10 - 0s - loss: 384.8664
Epoch 18/50
10/10 - 0s - loss: 371.1833
Epoch 19/50
10/10 - 0s - loss: 357.6287
Epoch 20/50
10/10 - 0s - loss: 345.0569
Epoch 21/50
10/10 - 0s - loss: 333.9144
Epoch 22/50
10/10 - 0s - loss: 322.2949
Epoch 23/50
10/10 - 0s - loss: 312.7018
Epoch 24/50
10/10 - 0s - loss: 301.9428
Epoch 25/50
10/10 - 0s - loss: 293.1437
Epoch 2

In [31]:
loss = model.evaluate(X_test, y_test)
y_hat = model.predict(X_test)
loss

10/10 [==============================] - 0s 1ms/step - loss: 177.0054


177.00538635253906

Create a list of mean squared errors and the standard deviation of the mean squared errors

In [32]:
total_mean_squared_errors = 50
mean_squared_errors = []
for i in range(0,total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)
    model.fit(X_train, y_train, epochs=50, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_hat = model.predict(X_test)
    errors = mean_squared_error(y_test, y_hat)
    mean_squared_errors.append(errors)
    
mean = np.mean(np.array(mean_squared_errors))
STD = np.std(mean_squared_errors)
print("Mean: ",str(mean), "Standard Deviation:",str(STD))

MSE 1: 131.5446319580078
MSE 2: 122.33574676513672
MSE 3: 112.80696105957031
MSE 4: 74.2048568725586
MSE 5: 61.26859664916992
MSE 6: 55.93060302734375
MSE 7: 55.07554626464844
MSE 8: 52.343658447265625
MSE 9: 51.94218826293945
MSE 10: 50.941566467285156
MSE 11: 51.906185150146484
MSE 12: 50.847328186035156
MSE 13: 51.72291946411133
MSE 14: 51.772430419921875
MSE 15: 50.92643356323242
MSE 16: 50.80622482299805
MSE 17: 50.05009078979492
MSE 18: 49.843929290771484
MSE 19: 50.41615295410156
MSE 20: 50.70391082763672
MSE 21: 55.38957595825195
MSE 22: 49.33818817138672
MSE 23: 48.38225173950195
MSE 24: 45.32906723022461
MSE 25: 44.847164154052734
MSE 26: 47.198490142822266
MSE 27: 44.81420135498047
MSE 28: 44.1415901184082
MSE 29: 46.40299606323242
MSE 30: 44.25127029418945
MSE 31: 47.15901184082031
MSE 32: 45.03095245361328
MSE 33: 47.849647521972656
MSE 34: 45.3390998840332
MSE 35: 43.06865692138672
MSE 36: 43.00734329223633
MSE 37: 43.641014099121094
MSE 38: 43.23373031616211
MSE 39: 43.3

In [33]:
experiment_name='Experiment'
MMSE='MMSE'
Standard_Dev='STD'
report_header = [experiment_name, MMSE,Standard_Dev]
def get_report(name_of_experiment, mean, STD):
    
    values = [[experiment_name, mean, STD]]

    return pd.DataFrame(columns=report_header, data=values)

experiment_name='Experiment2'

get_report(experiment_name, mean, STD)

,Experiment,MMSE,STD
0,Experiment2,52.366951,18.6767


We see that both standard deviation and mean increase the performance when we work with normalized data